In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pandas'

In [ ]:
path = 'ex2data1.txt'
data = pd.read_csv(path, header=None, names=['Exam 1', 'Exam 2', 'Admitted'])
data.head()

In [ ]:
positive = data[data['Admitted'].isin([1])]
negative = data[data['Admitted'].isin([0])]

fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(positive['Exam 1'], positive['Exam 2'], s=50, c='b', marker='o', label='Admitted')
ax.scatter(negative['Exam 1'], negative['Exam 2'], s=50, c='r', marker='x', label='Not Admitted')
ax.legend()
ax.set_xlabel('Exam 1 Score')
ax.set_ylabel('Exam 2 Score')
plt.show()

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
nums = np.arange(-10, 10, step=1)
nums = np.asmatrix(nums)

fig, ax = plt.subplots(figsize=(12,8))
ax.plot(nums.T, sigmoid(nums.T), 'r')
plt.show()

In [ ]:
def cost_function(theta, X, y):
    theta = np.asmatrix(theta)
    X = np.asmatrix(X)
    y = np.asmatrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log((1 - sigmoid(X * theta.T))))
    
    return np.sum((1./len(X)) * (first - second))

In [ ]:
def gradient_function(theta, X, y):
    theta = np.asmatrix(theta)
    X = np.asmatrix(X)
    y = np.asmatrix(y)
    
    return (1./len(X)) * np.dot(X.T, (sigmoid(X * theta.T) - y))


In [ ]:
data.insert(0, "Ones", 1)
data.head()

In [ ]:
cols = data.shape[1]
X = data.iloc[:,0:cols-1]
y = data.iloc[:,cols-1:cols]
X = np.array(X.values)
y = np.array(y.values)
theta = np.array(np.zeros(3))

In [ ]:
X.shape, theta.shape, y.shape

In [ ]:
cost_function(theta, X, y)

In [ ]:
gradient_function(theta, X, y)

In [ ]:
import scipy.optimize as opt
result = opt.fmin_tnc(func=cost_function, x0=theta, fprime=gradient_function, args=(X, y))
result

In [ ]:
def predict(theta, X):
    theta = np.asmatrix(theta)
    X = np.asmatrix(X)
    
    probability = sigmoid(X * theta.T)
    return [1 if x >= 0.5 else 0 for x in probability]

In [ ]:
theta_min = result[0]
predictions = predict(theta_min, X)
correct = [1 if (x == 0 and y == 0) or (x == 1 and y == 1) else 0 for (x, y) in zip(predictions, y)]
accuracy = (sum(map(int, correct)) % len(correct))
print ('accuracy = {0}%'.format(accuracy))


In [ ]:
data_path2 = "ex2data2.txt"
data2 = pd.read_csv(data_path2, header=None, names=['Test 1', 'Test 2', 'Accepted'])
data2.head()

In [ ]:
positive = data2[data2['Accepted'].isin([1])]
negative = data2[data2['Accepted'].isin([0])]

fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(positive['Test 1'], positive['Test 2'], s=50, c='b', marker='o', label='Accepted')
ax.scatter(negative['Test 1'], negative['Test 2'], s=50, c='r', marker='x', label='Rejected')
ax.legend()
ax.set_xlabel('Test 1 Score')
ax.set_ylabel('Test 2 Score')
plt.show()

In [ ]:
loc = data2.shape[1]
X = data2.iloc[:, 0:loc - 1]
y = data2.iloc[:, loc - 1: loc]
X = np.array(X.values)
y = np.array(y.values)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly=PolynomialFeatures(degree=6)
poly_X=poly.fit_transform(X)
poly_X.shape[1]

In [ ]:
def cost_function_with_reg(theta, X, y, rate):
    theta = np.asmatrix(theta)
    X = np.asmatrix(X)
    y = np.asmatrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log((1 - sigmoid(X * theta.T))))
    reg = (float(rate)/2 * len(X)) * theta[:, 1:theta.shape[1]] * theta[:, 1:theta.shape[1]].T
    
    return np.sum(first - second) / len(X) + reg

In [ ]:
theta = np.array(np.zeros(poly_X.shape[1]))
cost_function_with_reg(theta, poly_X, y, 0)

In [ ]:
def gradient_function_with_reg(theta, X, y, rate):
    theta = np.asmatrix(theta)
    X = np.asmatrix(X)
    y = np.asmatrix(y)
    gradient_reg = (float(rate) / len(X)) * theta[:,1:theta.shape[1]]
    regularized_term = np.concatenate([np.asmatrix([0]), gradient_reg.T])
    
    return gradient_function(theta, X, y) + regularized_term

In [ ]:
learning_rate = 5

In [ ]:
gradient_function_with_reg(theta, poly_X, y, learning_rate)

In [ ]:
result = opt.fmin_tnc(func=cost_function_with_reg, x0=theta, fprime=gradient_function_with_reg, args=(poly_X, y, learning_rate))
result

In [ ]:
theta_min = result[0]
predictions = predict(theta_min, poly_X)
correct = [1 if (x == 0 and y == 0) or (x == 1 and y == 1) else 0 for (x, y) in zip(predictions, y)]
accuracy = (sum(map(int, correct)) % len(correct))
print ('accuracy = {0}%'.format(accuracy))

In [ ]:
def find_decision_boundary(density, power, theta, threshhold):
    theta = np.asmatrix(theta)
    
    t1 = np.linspace(-1, 1.5, density)
    t2 = np.linspace(-1, 1.5, density)
    
    cordinates = [(x, y) for x in t1 for y in t2]
    x_cord, y_cord = zip(*cordinates)
    X = np.vstack((x_cord, y_cord)).T
    poly_X = feature_mapping(x_cord, y_cord, power)

    inner_product = np.asmatrix(poly_X.values) * theta.T

    decision = poly_X[np.abs(sigmoid(inner_product) - 0.5) < threshhold]

    return decision.f10, decision.f01

In [ ]:
def feature_mapping(x, y, power, as_ndarray=False):
#     """return mapped features as ndarray or dataframe"""
    # data = {}
    # # inclusive
    # for i in np.arange(power + 1):
    #     for p in np.arange(i + 1):
    #         data["f{}{}".format(i - p, p)] = np.power(x, i - p) * np.power(y, p)

    data = {"f{}{}".format(i - p, p): np.power(x, i - p) * np.power(y, p)
                for i in np.arange(power + 1)
                for p in np.arange(i + 1)
            }

    if as_ndarray:
        return pd.DataFrame(data).as_matrix()
    else:
        return pd.DataFrame(data)


In [ ]:
def feature_mapped_logistic_regression(X, y, power, l):
    X = np.asmatrix(X)
    y = np.asmatrix(y)
    
    poly=PolynomialFeatures(degree=power)
    poly_X=poly.fit_transform(X)
    theta = np.asmatrix(np.zeros(poly_X.shape[1]))
    
    result = opt.fmin_tnc(func=cost_function_with_reg, x0=theta, fprime=gradient_function_with_reg, args=(poly_X, y, learning_rate))
    
    return result
    

In [ ]:
def draw_boundary(power, l):
#     """
#     power: polynomial power for mapped feature
#     l: lambda constant
#     """
    density = 1000
    threshhold = 1 * pow(10, -7)

    data_path2 = "ex2data2.txt"
    data2 = pd.read_csv(data_path2, header=None, names=['Test 1', 'Test 2', 'Accepted'])
    loc = data2.shape[1]
    X = data2.iloc[:, 0:loc - 1]
    y = data2.iloc[:, loc - 1: loc]
    X = np.array(X.values)
    y = np.array(y.values)

    final_theta = feature_mapped_logistic_regression(X, y, power, l)
    px, py = find_decision_boundary(density, power, final_theta[0], threshhold)

    positive = data2[data2['Accepted'].isin([1])]
    negative = data2[data2['Accepted'].isin([0])]

    fig, ax = plt.subplots(figsize=(12,8))
    ax.scatter(positive['Test 1'], positive['Test 2'], s=50, c='b', marker='o', label='Accepted')
    ax.scatter(negative['Test 1'], negative['Test 2'], s=50, c='r', marker='x', label='Rejected')
    ax.scatter(px, py, c='g', s=2)
    ax.legend()
    ax.set_xlabel('Test 1 Score')
    ax.set_ylabel('Test 2 Score')
   
    plt.show()

In [ ]:
draw_boundary(power=1, l=0)

In [ ]:
draw_boundary(power=2, l=0)

In [ ]:
draw_boundary(power=6, l=0)

In [ ]:
draw_boundary(power=6, l=5)

In [ ]:
draw_boundary(power=6, l=100)